In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

from enum import Enum

def count_happy(model):
    happy = 0
    for agent in model.schedule.agents:
        if agent.happy:
            happy += 1
    return happy

class Color(Enum):
    RED = 1 # minority color
    BLUE = 2 # majority color

class Schelling(Model):
    """
    Model class for the Schelling segregation model.
    
    Parameters
    ----------
    height : int
             height of grid
    width  : int
             height of width
    density : float
            fraction of grid cells kept empty
    minority_fraction : float
            fraction of agent of minority color
    tolerance_threshold : int
    seed : int, optional
    
    Attributes
    ----------
    height : int
    width : int
    density : float
    minority_fraction : float
    schedule : RandomActivation instance
    grid : SingleGrid instance
    
    """

    def __init__(self, height=20, width=20, density=0.8, minority_fraction=0.2,
                 tolerance_threshold=4, seed=None):
        super().__init__(seed=seed)
        self.height = height
        self.width = width
        self.density = density
        self.minority_fraction = minority_fraction
        self.running = True
                
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus=True)
        self.datacollector = DataCollector(model_reporters={"happy":count_happy})
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if self.random.random() < self.density:             
                if self.random.random() > self.minority_fraction:
                    agent_color = Color.BLUE
                else:
                    agent_color = Color.RED
                    
                agent = SchellingAgent((x,y), self, agent_color, tolerance_threshold)
                self.grid.position_agent(agent, x, y)
                self.schedule.add(agent)
                    
            

    def step(self):
        """
        Run one step of the model.
        """
        
        self.schedule.step()
        self.datacollector.collect(self)
        if count_happy(self) == self.schedule.get_agent_count():
            self.running = False
        


class SchellingAgent(Agent):
    """
    Schelling segregation agent
    
    Parameters
    ----------
    pos : tuple of 2 ints
          the x,y coordinates in the grid
    model : Model instance
    color : {Color.RED, Color.BLUE}
    tolerance_threshold : int
    
    """

    def __init__(self, pos, model, color, tolerance_threshold):
        super().__init__(pos, model)
        self.pos = pos
        self.color = color
        self.tolerance_threshold = tolerance_threshold
        self.happy = True

    def meets_threshold(self,pos):
        counter = 0
        for neighbor in self.model.grid.iter_neighbors(pos, moore = True, include_center = False, radius = 1):
            if self.color != neighbor.color:
                counter += 1
                
        return counter <= self.tolerance_threshold
            
    def move_to_empty(self):
        for cell in self.model.grid.empties:
            if self.meets_threshold(cell):
                self.model.grid.move_agent(self, cell)
    
    def step(self):
        if not self.meets_threshold(self.pos):
            self.move_to_empty()
            self.happy = False
        else:
            self.happy = True
                
            
        

In [9]:
from mesa.batchrunner import batch_run

params = {'width': 20, 'height': 20,
          'density': [0.8, 0.825, 0.85, 0.875, 0.9],
          'minority_fraction': [0.2, 0.3, 0.4],
          'tolerance_threshold': [2, 3, 4]}

results = batch_run(
    Schelling,
    params,
    iterations=5,
    max_steps=50
)


100%|██████████| 225/225 [00:48<00:00,  4.63it/s]


In [11]:
results_df = pd.DataFrame(results)
results_df.head(50)

,RunId,iteration,Step,width,height,density,minority_fraction,tolerance_threshold,happy
0,0,0,50,20,20,0.800,0.2,2,307
1,1,0,50,20,20,0.800,0.2,3,296
2,2,0,5,20,20,0.800,0.2,4,311
3,3,0,50,20,20,0.800,0.3,2,316
4,4,0,50,20,20,0.800,0.3,3,314
5,5,0,50,20,20,0.800,0.3,4,322
6,6,0,3,20,20,0.800,0.4,2,305
7,7,0,3,20,20,0.800,0.4,3,301
8,8,0,3,20,20,0.800,0.4,4,315
9,9,0,50,20,20,0.825,0.2,2,298
